In [1]:
import json
import numpy as np
import os
import pandas as pd
import time

cwd = os.getcwd()
pkg_dir = os.path.dirname(cwd)
data_dir = os.path.join(pkg_dir, 'tests/data')

from reV.handlers.transmission import TransmissionFeatures
from reV.supply_curve.supply_curve import SupplyCurve

solar_fcr = 0.096
wind_fcr = 0.079

In [2]:
path = os.path.join(data_dir, 'sc_tables', '2_ri_agg_points_cache_064_sj_infsink.csv')
sc_table = pd.read_csv(path)

sc_gids = sc_table[['sc_point_gid', 'sc_point_row_id', 'sc_point_col_id']].drop_duplicates()
sc_gids = sc_gids.rename(columns={'sc_point_gid': 'gid',
                                  'sc_point_row_id': 'sc_row_ind',
                                  'sc_point_col_id': 'sc_col_ind'})

path = os.path.join(data_dir, 'sc_tables', '5_ri_subset_rural_pv_conus_multiyear.csv')
sc_results = pd.read_csv(path)
sc_results = sc_results.merge(sc_gids, on='gid').sort_values(['sc_row_ind', 'sc_col_ind'])



path = os.path.join(data_dir, 'sc_out', 'baseline_agg_summary.csv')
sc_points = pd.read_csv(path).rename(columns={'sc_gid': 'sc_point_gid'})
sc_points['sc_gid'] = sc_points.index

multipliers = sc_results[['gid', 'transmission_multiplier']].rename(columns={'gid': 'sc_point_gid'})

In [3]:
# Full sort
ts = time.time()
sc_full = SupplyCurve.full(sc_points, sc_table, fcr=0.096, sc_features=multipliers)
print('Full Supply Curve = {:.4f} seconds'.format(time.time() - ts))

# Simple
ts = time.time()
sc_simple = SupplyCurve.simple(sc_points, sc_table, fcr=0.096, sc_features=multipliers)
print('Simple Supply Curve {:.4f} seconds'.format(time.time() - ts))

# SC v1
ts = time.time()
sc_vintage = SupplyCurve.full(sc_points, sc_table, fcr=0.096, line_limited=True, sc_features=multipliers)
print('Vintage Supply Curve = {:.4f} seconds'.format(time.time() - ts))

Full Supply Curve = 1.6877 seconds
Simple Supply Curve 1.2528 seconds
Vintage Supply Curve = 1.5509 seconds


In [4]:
sc_vintage.head()

,area_sq_km,capacity,gen_gids,gid_counts,latitude,longitude,mean_cf,mean_lcoe,mean_res,pct_slope,...,sc_point_gid,sc_row_ind,sc_gid,transmission_multiplier,trans_gid,trans_type,lcot,total_lcoe,dist_mi,trans_cap_cost
0,2.3409,84.2724,[7],[289],41.993,-71.807999,0.124,124.002823,3.991000,3.195715,...,1,0,0,2.25,43636,TransLine,7.95737,131.96,9.215849,90037.665010
1,1.8954,68.2344,[7],[234],41.993,-71.737999,0.124,124.002823,3.991000,3.121439,...,2,0,1,2.25,43451,TransLine,1.58331,125.586,0.474527,17915.203811
2,8.0595,290.1420,"[22, 40]","[719, 276]",41.993,-71.737999,0.124,123.848488,4.005335,2.845650,...,2,0,2,2.25,80843,PCALoadCen,16.0604,139.909,22.025019,181722.924107
3,4.8438,174.3768,[66],[598],41.993,-71.668999,0.124,124.039299,3.996000,2.908158,...,3,0,3,2.25,80843,PCALoadCen,14.3803,138.42,19.720990,162712.961689
4,3.3696,121.3056,"[40, 76]","[356, 60]",41.993,-71.668999,0.124,123.855751,4.003163,2.859689,...,3,0,4,2.25,68810,Substation,0.85944,124.715,1.178627,9724.557271


In [5]:
sc_results.head()

,gid,line_join_gid,full_lcoe,subjoin,cap_cost_trans,trans_lcoe,category,onshore_dist,demreg,interconnect,...,sq_km,transreg,transmission_multiplier,region_sub_cost_mult,capacity,off_cab,dist_mi,geom,sc_row_ind,sc_col_ind
1,1,43413,139.954222,-999,34365.398376,2.228521,TransLine,2.468309,31,31,...,3.7827,31,2.25,1,121.0464,0,2.468309,0101000020E61000007F3F316DB4F351C05581361618FF...,0,1
16,2,80843,150.622094,-999,199559.252278,12.937095,PCALoadCen,22.489986,31,31,...,6.0507,31,2.25,1,193.6224,0,22.489986,0101000020E6100000FB475556B0F351C00613BCD174F8...,0,2
17,3,80843,149.879186,-999,195722.924107,12.646605,PCALoadCen,22.025019,31,31,...,16.8561,31,2.25,1,539.3952,0,22.025019,0101000020E6100000661CB87F41EF51C053369E811DFF...,0,3
18,4,80843,148.431359,-999,176712.961689,11.401254,PCALoadCen,19.720990,31,31,...,14.2803,31,2.25,1,456.9696,0,19.720990,0101000020E610000039CA1C90CEEA51C0279CEA8B21FF...,0,4
19,5,80843,148.543712,-999,175680.092509,11.348354,PCALoadCen,19.595806,31,31,...,16.5159,31,2.25,1,528.5088,0,19.595806,0101000020E6100000308436553EEF51C0B91AA23A7AF8...,0,5
